In [ ]:
import pandas as pd

import openai
import time
delay_sec = 5
# remember to set your open AI API key!
openai.api_key = '' #replace it with your own API

import numpy as np
import pickle

EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed = np.zeros(shape=(len(gene_all),EMBED_DIM))

def get_gpt_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return np.array(openai.Embedding.create(input = [text], model=model)['data'][0]['embedding'])

In [ ]:
gene_name_to_GPT_response = {}
gene_name_getembedding = {}

In [ ]:
df = pd.read_csv("gpt_ncbi_allgene.csv", index_col = 0) # load gene name, modify this pathway is acceptable. If the name is in ensemble id format, using mygene to transfer the format.
gene_all = list(df['gene'].values)

In [ ]:
gene_completion_test = gene_all
for gene in gene_completion_test:
    print(gene)
    try:
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo-1106", 
                    messages=[{"role": "user", 
                               "content": f"Please summarize the major function of gene: {gene}. Use academic language in one paragraph and include pathway information."}]) # replace the prompt for different metadata.
        gene_name_to_GPT_response[gene] = completion.choices[0].message.content
        gene_name_getembedding[gene] = get_gpt_embedding(gene_name_to_GPT_response[gene])
        time.sleep(1)
    except (openai.APIError, 
                    openai.error.APIError, 
                    openai.error.APIConnectionError, 
                    openai.error.RateLimitError, 
                    openai.error.ServiceUnavailableError, 
                    openai.error.Timeout) as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        pass

In [ ]:

with open('ensem_describe.pickle', 'wb') as handle:
    pickle.dump(gene_name_to_GPT_response, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ensem_emb_35.pickle', 'wb') as handle:
    pickle.dump(gene_name_getembedding, handle, protocol=pickle.HIGHEST_PROTOCOL)